<a href="https://colab.research.google.com/github/AleksxanderAlekseyuk/Deal_with_epicrisis/blob/main/treatment_initiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import os
import pandas as pd
import numpy as np
from glob import glob
from re import sub

In [ ]:
lst = list(glob(os.path.join('/content/drive/MyDrive/phd/script.zip (Unzipped Files)/script', '*')))
ids = []
birthday = []
treatment = []

In [ ]:
for doc in lst:
   with open(doc, encoding='utf-8', errors='ignore') as f:
     file_ = sub('<|>|\\(|\\)|меньше|меньшн|больше|более|менее|iultra|\\(|ручная|\\)|\\(|автоматическая|\\)', 
                  '.', ''.join(f.read().split()).lower())
     file_ =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})', 
                     '\\1-\\2-\\3', file_)
     file_ = sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})',
                    '\\1-\\2-20\\3', file_)
     id = re.findall('\d+', file_)[0]
     birthday_ = re.findall('\d{2}-\d{2}-\d{4}', file_)[0]
     treatment_ = re.findall("(?<=проведенноелечение:).*?(?=рекомендации|наблюдение)", 
                            file_)
     ids.append(id)
     birthday.append(birthday_)
     treatment.append(treatment_)

In [216]:
# задаём словарь, в котором ключ - истинное название препарата, а значения - варианты названий ( коммерческих) и
# названия с ошибками для обработки с помощью регулярных выражений

dict_treatment = {
    'лизиноприл':'л.з.....ил|лизитар|лизинеоприл|лизаретик', 
    'цефтриаксон':'три.....ф|цефт.....он|цефтриакосн|ceftriaxon', 
    "метопролол":"мет.....ол|эгилок", 
    'тейкопланин': "тей......ин.",
    "амброксол": "аброкол|амброко.|амб......|амброксолс\
            |амброкосл|амброксолл|амброксолл|аброксол|амброксолл|ambroxoli", 
    "моксифлоксацин": 'пле....кс.|мокс.......цин',
    "молсидомин":"м.лсид...н",
    "ривароксабан": "xarelto|ксарелт.",
    "спиронолактон":"сп.р.......он.|верош....н",
     "бисопролол":"б.с.пр..ол|бисопралдол|бикард|конкор", 
     "лансопрозол": "лан......ол|ланс.зол",
     "меропенем":"мер....ем",
     "аторвастатин": "липромак|аторв.......",
      "дексаметазон": "декс.......н|дексаетазон",
                 "левофлоксацин": "лев.......цин",
                 "омепразол": "омез|омепр..ол|omeprazoli",
                 "азитромицин": "азит.......|azitromycini",
                 "ацетилцистеин": "ацецемед|асс|асс|ацц|ацемед|ацецезон|aццпор",
                 "кандесартан": "к.с.рк",
                 "амоксициллин": "амоксицилин|амо........н",
                 "озельтамивир": "флустоп|озел.......р",
                 "эноксапарин": "клексан|кл...ан",
                 "имепенем": "им....ем",
                 "клопидогрел": "клопидогрель|плавикс|клорпидогрель",
                 "аспикард": "аспкиард|кардиомагнил|ас....рд",
                 "амлодипин":"амл....ин|норваск",
                 "амикацин": "ам.к...н",
                 "беродуал": "бер.дуал|ипратропия.бромид",
                 "парацетамол": "парац.....л",
                 "гидроксихлорохин": "плаквенил|имар.|иммар.",
                 "метилпреднизолон": "медрол|м.дрол|метипред|диметилпреднизолон",
                 "теофиллин": "теофилин|теотард|теофилл",
                 "кларитромицин": "кларибацин|кла.......цин",
                 "лозартан": "лазртан|л...ртан|л.риста",
                 "валсартан": "валозартан|валз|вал....ан",
                 "моксонидин": "м.кс.н.дин",
                 "амиодарон": "ами.д.р.н|к.рд.рон",
                 "полимиксин": "колистин|колистат|к.л.стат",
                 "преднизолон": "преднезолон|пре.н.з.лон",
                 "рамиприл": "рам.л...|рам.прил",
                 "карбомазепин": "карб.м.з.п.н",
                 "кетаролак": "кет.р.лак|кет.рол|кет.н.в",
                 "гентамицин": "г.нт.мицин",
                 "цефепим": "ц.ф.пим",
                 "цефоперазон/сульбактам": "цеф.пер.зон|цеф.пер.зон.су.......",
                 "тайгециклин": "тиг.циклин|та.г.циклин",
                 "зопиклон": "з.п.клон|соне.|соне.с|сомнол",
                  "нандропарин": "фраксипарин|фр.ксип.рин",
                  "далтепарин": "фрагмин",
                  "эноксапарин": "кл.ксан",
                  "кислород": "кислород..ерапия|увло2|кислороднаяподдержка.увлажненныйкислород|увл.кислород",
                  "гепарин": "heparini|гепаин",
                  "индапамид": "индапамидафон|индапафон|индап|инд.....д",
                  "": r'в\\в|п/к|свой|витс|ммольл|крови|ммольл|время|вв|п\\к|в\\в|п//к|\\в|п\\к|12мгвнутривеннокапна100.0физ.р-ра5дней|0.03.по1таб3разавсутки|i2,0в\\вкапельнона100,0nacl0.9%'


    }


In [ ]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in \
                     stopwords])
df["Treatment"] = df["Treatment"].apply(lambda text: remove_stopwords(text))

In [217]:
cleaned_treatment = []

for lists in treatment:
  for words in lists:
    try:
        for i, j in dict_treatment.items():
            words = re.sub(j, i, words)
        cleaned_treatment.append(words)
    except:
        cleaned_treatment.append("None")


In [ ]:
re.sub(r"в\\в", '', "в\\в")

''

In [218]:
cleaned_treatment

['пронпозиция,кислород,дексаметазон,омепразол,ацетилцистеин,далтепарин,бисопролол,ко-амлесса,нимесулид,моноинсулин,метилпреднизолон.',
 'инфузионнаятерапия,витаминотерапия,гепарин,дексаметазон,нистатин,ацетилцистеинзон,омепразол,ранитидин,флуконазолтрансфузионнаятерапия:непроводилась',
 'бортезамиб,циклофосфан,дексаметазон,фуросемид,золендркислота,боривит,бисопролол,ацикловир,ко-тримаксазол,левофлоксацинкап,флуканазолкап,фамоцидин,гепарин,метопролол,метилпреднизолон,далтепарин,омепразол,парацетамоксициллинемкап,имепенемкап,пронпозиц,инсуфлкислород',
 'пронпозиция,дексаметазон,омепразол,нандропарин,ацетилцистеин,парацетамол,зопиклон.',
 'дексаметазон,омепразол,гепарин,парацетамол,глюкоза.цефтриаксон,метформин,кандесартан,карведилол.гликлазид,сульпирид,амитриптиллин,метилпреднизолон,кислород,пронпозиция.',
 'метилпреднизолон,дексаметазон,ривароксабан,,парацетамол,цефтриаксонкап,,прон-позиция,инсуффляцияо2',
 'дексаметазон,омепразол,далтепарин,эутирокс,бисопролол,аторвастатин,дилтиазем.ва

In [ ]:
data_treatment = {'ID':ids, 'birthday':birthday,'treatment':treatment}
df = pd.DataFrame(data_treatment)
#df.replace(r'', np.nan, regex=True)

In [ ]:
df.head()

In [ ]:
df["Treatment"] = df["treatment"].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

In [ ]:
df["Treatment"] = df["Treatment"].values.astype('str')

In [ ]:
df["Treatment"] = df["Treatment"].apply(lambda x: treatment_cleaning)

In [ ]:
import re
import nltk
import string


In [ ]:
from collections import Counter
cnt = Counter()
for text in df["Treatment"]:
    for word in text.split(","):
        cnt[word] += 1
        
cnt.most_common(100)

[('<function treatment_cleaning at 0x7f1f18d52e60>', 1147)]

[('омепразол', 519),
 ('дексаметазон', 478),
 ('фрагмин', 364),
 ('гепарин', 316),
 ('парацетамол', 313),
 ('цефтриаксон', 295),
 ('', 254),
 ('аспикард', 230),
 ('амброксол', 214),
 ('метилпреднизолон', 163),
 ('пронпозиция', 162),
 ('флуконазол', 161),
 ('ацц', 159),
 ('ацецезон', 156),
 ('бисопролол', 148),
 ('прон-позиция', 136),
 ('преднизолон', 129),
 ('аллеркапс', 129),
 ('фам.идин', 128),
 ('инфузионнаятерапия', 122),
 ('лизиноприл', 115),
 ('фраксипарин', 110),
 ('зопиклон', 102),
 ('аторвастатин', 99),
 ('метопролол', 94),
 ('кларитромицин', 89),
 ('моноинсулин', 88),
 ('левофлоксацин', 88),
 ('кислород.ерапия', 83),
 ('лозартан', 83),
 ('амлодипин', 79),
 ('меропенем', 78),
 ('моксонидин', 73),
 ('омепрозол', 70),
 ('урсаклин', 70),
 ('азитромицин', 63),
 ('кпс', 56),
 ('лактулоза', 52),
 ('молсидомин', 50),
 ('метронидазол', 49),
 ('ванкомицин', 48),
 ('фуросемид', 46),
 ('цефепим', 42),
 ('пароксетин', 39),
 ('димедрол', 38),
 ('клопидогрель', 37),
 ('нистатин', 36),
 ('ац

In [ ]:
treatment


[['пронпозиция,кислород.ерапия,дексаметазон,омепразол,ацц,фрагмин,бисопролол,ко-амлесса,нимесулид,моноинсулин,метилпреднизолон.'],
 ['инфузионнаятерапия,витамин.ерапия,гепарин,дексаметазон,нистатин,аццзон,омепразол,ранитидин,флуконазолтрансфузионнаятерапия:непроводилась'],
 ['бортезамиб,циклофосфан,дексаметазонв\\в,фуросемид,золендркисл.а,боривит,бисопролол,ацикловир,ко-тримаксазол,левофлоксацинв\\вкап,флуканазолв\\вкап,фамоцидин,гепарин,метопролол,метилпреднизолон,фрагмин,омепразол,парацетамол,меропинемв\\вкап,имиценемв\\вкап,пронпозиц,инсуфлувло2'],
 ['пронпозиция,дексаметазон,омепразол,фраксипарин,ацецезон,парацетамол,зопиклон.'],
 [],
 ['дексаметазонв\\в,омепрозол,гепаринп\\к,парацетамол,глюкозав\\в.цефтриаксонв\\в,метформин,косарк,карведилол.гликлазид,сульпирид,амитриптиллин,метилпреднизолон,кислород.ерапия,пронпозиция.'],
 [],
 ['метилпреднизолон,дексаметозон,ксарелто,витс,парацетамол,цефтриаксонв\\вкап,,прон-позиция,инсуффляцияо2'],
 ['дексаметазон,омепразол,фрагмин,эутирокс,бис